<a href="https://colab.research.google.com/github/ANIK8-JHA/Minor_Project/blob/main/Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [106]:
# Gathering Data using Bing Downloader

In [ ]:
!pip install bing-image-downloader

In [ ]:
!mkdir image_data

In [109]:
from bing_image_downloader import downloader

In [ ]:
downloader.download("tejas fighter jet",limit=20,output_dir="image_data",adult_filter_off=True)

In [ ]:
downloader.download("f35 fighter jet",limit=20,output_dir="image_data",adult_filter_off=True)

In [ ]:
downloader.download("rafael fighter jet",limit=20,output_dir="image_data",adult_filter_off=True)

In [ ]:
# Preprocessing Data

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize

In [ ]:
target_data = []
image_data = []
flat_data = []

DATADIR = '/content/image_data'
Categories = ['tejas fighter jet','f35 fighter jet','rafael fighter jet']

In [ ]:
#Resizing and Flattening

for cat in Categories:
  class_num = Categories.index(cat) # Label Encoding the values
  path = os.path.join(DATADIR,cat) # Create path to use all the images
  for img in os.listdir(path):
    img_array = imread(os.path.join(path,img))
    img_resized = resize(img_array,(150,150,3))
    flat_data.append(img_resized.flatten())
    image_data.append(img_resized)
    target_data.append(class_num)

flat_data = np.array(flat_data)
target_data = np.array(target_data)
image_data = np.array(image_data)

In [ ]:
# Hyperparameter Tuning

In [ ]:
#Split data into Training and Testing
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(flat_data,target_data,
                                  test_size=0.3,random_state=109)


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm
param_grid = [
              {'C':[1,10,100,1000],'kernel':['linear']},
              {'C':[1,10,100,1000],'gamma':[0.001,0.0001],'kernel':['rbf']},
]
svc = svm.SVC(probability=True)
clf = GridSearchCV(svc,param_grid)
clf.fit(x_train,y_train)

In [ ]:
y_pred = clf.predict(x_test)

In [ ]:
# Evaluation

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
accuracy_score(y_pred, y_test)

In [ ]:
confusion_matrix(y_pred, y_test)

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
confusion_matrix(y_pred,y_test)
accuracy_score(y_pred,y_test)

In [ ]:
# Saving model using pickel library

In [ ]:
import pickle
pickle.dump(clf, open('img_model.p', 'wb')) # wb - writing byte
model = pickle.load(open('img_model.p', 'rb')) # rb - reading byte

In [ ]:
# Prediction

In [ ]:
flat_data = []
url = input('Enter your URL')
img = imread(url)
img_resized = resize(img,(150,150,3))
flat_data.append(img_resized.flatten())
flat_data = np.array(flat_data)
print(img.shape)
plt.imshow(img_resized)
y_out = model.predict(flat_data)
y_out = Categories[y_out[0]]
print(f' PREDICTED OUTPUT: {y_out}')